# Generate the data for the ISDM-24 track

In [ ]:
%pip install -q pandas openpyxl jinja2

In [ ]:
# Print formatted dates in spanish locale
import locale
locale.setlocale(locale.LC_ALL, 'es_ES.UTF-8')

In [ ]:
# Read the dataframe from the .xlsx file given by Easychair
import pandas as pd
df = pd.read_excel('../info/ISDM2024.xlsx',index_col=0,header=0)

# df.Author to spanish 'and' -> 'y'
df['Authors'] = df['Authors'].str.replace(' and ', ' y ', regex=False)
df

In [ ]:
from dataclasses import dataclass
from datetime import datetime, timedelta

@dataclass
class Session:
    name: str
    start: datetime
    paperstart: datetime
    end: datetime
    location: str
    chair: str
    papers: list[int]

isdm1 = Session('ISDM-1',
                datetime(2024,7,18,9,15),
                datetime(2024,7,18,9,15)+timedelta(minutes=10), # 10 mins presentation
                datetime(2024,7,18,10,15),
                'Aula A.3.1',
                'Diego Sevilla Ruiz',
                [91, 94, 24])

isdm2 = Session('ISDM-2',
                datetime(2024,7,18,16,00),
                datetime(2024,7,18,16,00),
                datetime(2024,7,18,17,30),
                'Aula A.3.1',
                'Pedro José Clemente Martín',
                [3, 2, 4, 121, 124])

isdm3 = Session('ISDM-3',
                datetime(2024,7,18,18,00),
                datetime(2024,7,18,18,00),
                datetime(2024,7,18,19,00),
                'Aula A.3.1',
                'Diego Sevilla Ruiz y Pedro José Clemente Martín',
                [12, 57, 81])

sessions = {s.name : s for s in [isdm1, isdm2, isdm3]}

# No article left behind
assert set(df.index) == set().union(*list(map(lambda s: s.papers, sessions.values())))

In [ ]:
# Produce the slide index from the slideindex template
from jinja2 import Template

def gen_session_index(sessions: dict[str, Session]) -> None:
    template = Template(open('templates/sessionindex.tex.j2').read())
    with open('../pr/sessionindex.tex', 'w') as f:
        f.write(template.render(sessions=sessions))

gen_session_index(sessions)

In [ ]:
def gen_session_slides(sessions: dict[str, Session], df: pd.DataFrame) -> None:
    template = Template(open('templates/sessionslide.tex.j2').read())

    def get_duration_time(articletype: str):
        match articletype:
            case 'relevante' | 'completo':
                return 20
            case 'corto' | 'herramienta':
                return 10
            case _:
                raise ValueError(f'Unknown article type {articletype}')

    def get_duration(articletype: str):
        match articletype:
            case 'relevante' | 'completo':
                return f'{get_duration_time(articletype)}~min'
            case 'corto' | 'herramienta':
                return f'{get_duration_time(articletype)}~min'
            case _:
                raise ValueError(f'Unknown article type {articletype}')

    def time_str(d: datetime) -> str:
        return d.strftime('%H:%M')

    def date_str(d: datetime) -> str:
        return d.strftime('%A, %d de %B de %Y, %H:%M')

    for session_name, session in sessions.items():
        with open(f'../pr/{session_name}.tex', 'w') as f:
            f.write(template.render(session=session,
                                    papers=session.papers,
                                    date_str = date_str,
                                    time_str = time_str,
                                    df=df,
                                    get_duration=get_duration,
                                    get_duration_time=get_duration_time,
                                    timedelta=timedelta))

gen_session_slides(sessions, df)

In [ ]:
%pip install -q pyecore pyecoregen

In [ ]:
!pyecoregen -e isdm-track/model/IsdmTrack.ecore -o . --auto-register-package

In [ ]:
import IsdmTrack as isdm

track = isdm.Track(name="ISDM'24")

# Participants
i_participants: dict[str, isdm.Participant] = {}

i_authors: dict[str, isdm.Participant] = {}

# PC members
i_pcmembers: dict[str,isdm.Participant] = {
    "Fco. Javier Bermúdez": isdm.Participant(name="Fco. Javier Bermúdez", university="Universidad de Murcia"),
    "Juan Boubeta-Puig": isdm.Participant(name="Juan Boubeta-Puig", university="Universidad de Cádiz"),
    "Lola Burgueño": isdm.Participant(name="Lola Burgueño", university="Universidad de Málaga"),
    "Javier Luis Cánovas": isdm.Participant(name="Javier Luis Cánovas", university="Universitat Oberta de Catalunya"),
    "Carlos Cetina": isdm.Participant(name="Carlos Cetina", university="Universidad San Jorge"),
    "Robert Clarisó": isdm.Participant(name="Robert Clarisó", university="Universitat Oberta de Catalunya"),
    "Pedro José Clemente Martín": isdm.Participant(name="Pedro José Clemente Martín", university="Universidad de Extremadura"),
    "Javier Criado": isdm.Participant(name="Javier Criado", university="Universidad de Almería"),
    "Juan de Lara": isdm.Participant(name="Juan de Lara", university="Universidad Autónoma de Madrid"),
    "Félix Óscar García": isdm.Participant(name="Félix Óscar García", university="Universidad de Castilla-La Mancha"),
    "Abel Gómez": isdm.Participant(name="Abel Gómez", university="Universitat Oberta de Catalunya"),
    "David Granada": isdm.Participant(name="David Granada", university="Universidad Rey Juan Carlos"),
    "Esther Guerra": isdm.Participant(name="Esther Guerra", university="Universidad Autónoma de Madrid"),
    "Luis Iribarne": isdm.Participant(name="Luis Iribarne", university="Universidad de Almería"),
    "José Merseguer": isdm.Participant(name="José Merseguer", university="Universidad de Zaragoza"),
    "Xavier Oriol": isdm.Participant(name="Xavier Oriol", university="Universitat Politècnica de Catalunya"),
    "Vicente Pelechano": isdm.Participant(name="Vicente Pelechano", university="Universitat Politécnica de València"),
    "José Raúl Romero": isdm.Participant(name="José Raúl Romero", university="Universidad de Córdoba"),
    "Roberto Rodríguez-Echevarría": isdm.Participant(name="Roberto Rodríguez-Echevarría", university="Universidad de Extremadura"),
    "Francisco Ruiz": isdm.Participant(name="Francisco Ruiz", university="Universidad de Castilla-La Mancha"),
    "Jesús Sánchez-Cuadrado": isdm.Participant(name="Jesús Sánchez-Cuadrado", university="Universidad de Murcia"),
    "Diego Sevilla Ruiz": isdm.Participant(name="Diego Sevilla Ruiz", university="Universidad de Murcia"),
    "Ernest Teniente": isdm.Participant(name="Ernest Teniente", university="Universitat Politecnica de Catalunya"),
    "Javier Troya": isdm.Participant(name="Javier Troya", university="Universidad de Málaga"),
    "Juan Manuel Vara": isdm.Participant(name="Juan Manuel Vara", university="Universidad Rey Juan Carlos"),
    "Cristina Vicente Chicote": isdm.Participant(name="Cristina Vicente Chicote", university="Universidad de Extremadura")
}
for _, participant in i_pcmembers.items():
    i_participants[participant.name] = participant

# Papers and authors, traverse the df DataFrame
i_papers: dict[int, isdm.Paper] = {}
for index, row in df.iterrows():
    i_papers[index] = isdm.Paper(title=row['Title'],
                                 type=row['Tipo'],
                                 abstract=row['Abstract'],
                                 keywords=row['Keywords'])
    # Split and add authors
    authors = row['Authors']
    authors = authors.replace(' y ', ',')
    authors = authors.split(',')
    for author_name in authors:
        author_name = author_name.strip()

        # Author already in the list of participants?
        if author_name in i_participants:
            author = i_participants[author_name]
        else:
            author = isdm.Participant(name=author_name)
            i_participants[author_name] = author

        if author_name not in i_authors:
            i_authors[author_name] = author

        i_papers[index].authors.append(author)


i_sessions: dict[str, isdm.Session] = {}

for name, session in sessions.items():
    i_sessions[name] = isdm.Session(
        name=session.name,
        start=int(round(session.start.timestamp())),
        end=int(round(session.end.timestamp())),
        paperstart=int(round(session.paperstart.timestamp())),
        location=session.location
    )

    # Add chairs to the session
    for chair in session.chair.split(' y '):
        i_sessions[name].chairs.append(i_pcmembers[chair])

    # Add papers to the session
    for paperid in sessions[name].papers:
        i_sessions[name].papers.append(i_papers[paperid])

# Add participants to the track
for _, participant in i_participants.items():
    track.participants.append(participant)

# Add sessions to the track
for _, session in i_sessions.items():
    track.sessions.append(session)

# Add papers to the track
for _, paper in i_papers.items():
    track.papers.append(paper)

# Add authors to the track
for _, author in i_authors.items():
    track.authors.append(author)

# Add pc members to the track
for _, pc_member in i_pcmembers.items():
    track.pcmembers.append(pc_member)

In [ ]:
template = Template(open('templates/sessionslide-model.tex.j2').read())

for session in track.sessions:
    print(template.render(session=session))
